<a href="https://colab.research.google.com/github/Mahesh18190/Resume-Screener/blob/main/Resume_Screener_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q\
  langchain==0.1.16\
  langchain-community==0.0.34\
  langchain-core==0.1.45\
  chromadb \
  pypdf \
  gradio \
  langchain-google-genai \
  google-genai

In [4]:

!pip install -q sentence-transformers

# ==============================
# STEP 1: Imports
# ==============================
import os
import gradio as gr
from google.colab import userdata

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

from langchain_google_genai import ChatGoogleGenerativeAI

# HuggingFace Embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

# Agent
from langchain.agents import initialize_agent, AgentType
from langchain_core.tools import tool

# ==============================
# API Key (Gemini)
# ==============================
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE")

# ==============================
# LLM + Embeddings
# ==============================
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.2
)

# HF embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectordb = None  # global

# ==============================
# Process PDFs
# ==============================
def process_pdfs(resume_file, jd_file):
    global vectordb

    if resume_file is None or jd_file is None:
        return "⚠️ Upload both Resume and Job Description PDFs."

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = []

    # Resume
    resume_docs = PyPDFLoader(resume_file.name).load()
    for d in resume_docs:
        d.metadata["type"] = "RESUME"
        d.metadata["source"] = resume_file.name
    docs.extend(splitter.split_documents(resume_docs))

    # JD
    jd_docs = PyPDFLoader(jd_file.name).load()
    for d in jd_docs:
        d.metadata["type"] = "JD"
        d.metadata["source"] = jd_file.name
    docs.extend(splitter.split_documents(jd_docs))

    vectordb = Chroma.from_documents(
        documents=docs,
        embedding=embeddings
    )


    return f" Resume & JD processed. Total chunks: {len(docs)}. Now go to 'Ask' tab."

# ==============================
# Tool
# ==============================
@tool
def search_docs(query: str) -> str:
    """Search resume and JD content and return evidence."""
    global vectordb
    if vectordb is None:
        return "NO_DOCUMENTS: Please upload and process PDFs first."

    results = vectordb.as_retriever(search_kwargs={"k": 6}).invoke(query)
    if not results:
        return "NO_MATCHES: Nothing relevant found in the documents."

    return "\n\n---\n\n".join(
        f"[{r.metadata.get('type')}] {r.page_content}"
        for r in results
    )

# ==============================
# STEP 6: Agent
# ==============================
SYSTEM_RULES = """
You are a Resume Fit Scoring Agent.

TASK:
- Compare Resume vs Job Description
- Give Fit Score (0–100)
- Verdict (Strong / Moderate / Weak)
- Strengths
- Missing Skills / Gaps
- Resume Improvements
- Interview Prep Topics

RULES:
- Use search_docs BEFORE answering (collect evidence).
- Do NOT hallucinate skills/experience not present in the resume.
- If info is missing, say: "I don't see this information in the documents."
- Keep the output structured and simple.
"""

agent = initialize_agent(
    tools=[search_docs],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

# ==============================
# Chat
# ==============================
def resume_chat(message, history):
    return agent.run(f"{SYSTEM_RULES}\n\nUser question: {message}")

# ==============================
# UI
# ==============================
with gr.Blocks() as demo:
    gr.Markdown("# 📄 Resume Fit Scorer")

    with gr.Tab("Upload"):
        r = gr.File(label="Resume PDF", file_types=[".pdf"])
        j = gr.File(label="Job Description PDF", file_types=[".pdf"])
        b = gr.Button("Process", variant="primary")
        s = gr.Textbox(label="Status", lines=5, interactive=False)
        b.click(process_pdfs, [r, j], s)

    with gr.Tab("Ask"):
        gr.ChatInterface(
            fn=resume_chat,
            examples=[
                "Give me fit score (0-100) and missing skills.",
                "List strengths and gaps for this JD.",
                "How can I improve my resume for this role?",
                "What interview topics should I prepare based on missing skills?"
            ]
        )

demo.launch(share=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://33176ec12406b20ee8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from langchain import embeddings
!pip install -q sentence-transformers

# import the necessary packages

import os
import gradio as gr
from google.colab import userdata


from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI


#HuggingFaace Embeddings

from langchain.embeddings import HuggingFaceEmbeddings


#Agent

from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool



#API KEY

os.environ["GOOGLE_API_KEY"]=userdata.get("GOOGLE")

#LLM + Embeddings

llm=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.2
)

#Hf Embeddings

embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

Vectordb= None

#Process PDF's

def process_pdf(resume_file,jd_file):
  global Vectordb


  if resume_file is None or jd_file is None:
    return "Upload Both Resume and Job Description PDFS"


  splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
  docs=[]
 #Resume

  resume_docs = PyPDFLoader(resume_file.name).load()

  for d in resume_docs:
    d.metadata["type"]="RESUME"
    d.metadata["source"]=resume_file.name
  docs.extend(splitter.split_documents(resume_docs))


  #JD

  jd_docs = PyPDFLoader(jd_file.name).load()
  for d in jd_docs:
    d.metadata["type"]="JD"
    d.metadata["sorce"]=jd_file.name
  docs.extend(splitter.split_documents(jd_docs))


  Vectordb= Chroma.from_documents(
      documents=docs,
      embedding=embeddings
  )

  return f"Resume  & JD processed.Total chunks:{len(docs)}.Now go to 'Ask' tab"


  #tool
@tool
def search_docs(query:str) ->str:
  """Search resume and Jd content and return evidence."""
  global Vectordb
  if Vectordb is None:
    return "NO DOCUMENTS:please upload and process PDFs first"
  results=Vectordb.as_retriever(search_kwargs={"k":6}).invoke(query)
  if not results:
    return "NO MATCHES:nothing relevant found in the documents"

  return "\n\n---\n\n".join(
      f"[{r.metadata.get('type')}] {r.page_content}"
      for r in results
  )


  #Agent
SYSTEM_RULES="""
  You are a resume fit scoring agent.

  TASK:
  -Compare resume VS Job descripption
  -Give fir score(0-100)
  -Verdict(Strong/Moderate/Weak)
  -Strengths
  -Missings Skills/Gaps
  -Resume Improvements
  -Interview prep topics

  RULES:
  -Use Search_docs before answering (collect evidence).
  -Do not halluciate skills/experience not present in the resume .
  -if info is missing ,say: "I don't see this information in the documents."
  -Keep the output structured and simple .
  """

agent= initialize_agent(
    tools=[search_docs],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)


#chat

def resume_chat(message,history):
  return agent.run(f"{SYSTEM_RULES}\n\nUser question :{message}")


#UI

with gr.Blocks() as demo:
  gr.Markdown("# Resume Fit Scorer")

  with gr.Tab("Upload"):
    r= gr.File(label="Resume PDF",file_types=["pdf"])
    j= gr.File(label="Job Description PDF",file_types=["pdf"])
    b= gr.Button("Process",variant="primary")
    s= gr.Textbox(label="Status", lines=5,interactive=False)
    b.click(process_pdf,[r,j],s)

  with gr.Tab("Ask"):
    chatbot = gr.Chatbot(label="Chat History", height=400)
    msg = gr.Textbox(label="Your message", placeholder="Type your message here...")
    send_btn = gr.Button("Send")

    def chat_function_manual(message, history):
        if message.strip() == "":
            return "", history

        response = agent.run(f"{SYSTEM_RULES}\n\nUser question :{message}")
        history.append([message, response])
        return "", history

    msg.submit(chat_function_manual, [msg, chatbot], [msg, chatbot])
    send_btn.click(chat_function_manual, [msg, chatbot], [msg, chatbot])

    gr.Examples(
        examples=[
            "Give me fit score (0-100) and missing skills.",
            "List strength and gaps for this JD.",
            "How can i improve my resume for this role?.",
            "What interview topics should i prepare based on missing skills?."
        ],
        inputs=[msg],
        outputs=[msg]
    )

demo.launch()